In [61]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras import metrics, regularizers

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [62]:
#Load cleaned dataset
data = pd.read_csv('../../Results/Cleaned_JobDescs.csv', header = 0, names = ['Query', 'Description'])
#data = pd.read_csv('../../Results/Cleaned_JobsNonIT.csv', header = 0, names = ['Query', 'Description'])

In [63]:
#Split the dataset to Training and Test subsets (90/10)
train, test = train_test_split(data, test_size = 0.1, random_state = 17) #random_state = None

train_descs = train['Description']
train_labels = train['Query']
 
test_descs = test['Description']
test_labels = test['Query']

In [70]:
# Model Parameters
num_labels = len(train_labels.unique())
vocab_size = 1000
batch_size = 32
nb_epoch = 50

In [71]:
# Convert Texts to Numeric Vectors for Input
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(train_descs)
x_train = tokenizer.texts_to_matrix(train_descs, mode = 'tfidf') #count, tfidf, binary
x_test = tokenizer.texts_to_matrix(test_descs, mode = 'tfidf')

encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [72]:
model = Sequential()
model.add(Dense(2048, input_shape=(vocab_size,)))
model.add(Activation('tanh'))
model.add(Dropout(0.1))
#model.add(Dense(1024))
#model.add(Activation('tanh'))
#model.add(Dropout(0.1))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'sgd', #'sgd', 'adam', 'RMSprop', 'Adagrad'
              metrics = [metrics.categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 2048)              2050048   
_________________________________________________________________
activation_33 (Activation)   (None, 2048)              0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 25)                51225     
_________________________________________________________________
activation_34 (Activation)   (None, 25)                0         
Total params: 2,101,273
Trainable params: 2,101,273
Non-trainable params: 0
_________________________________________________________________


In [73]:
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = nb_epoch,
                    verbose = True,
                    validation_split = 0.2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/50
7200/7200 [==============================] - 7s 970us/step - loss: 2.2360 - categorical_accuracy: 0.3682 - val_loss: 1.6827 - val_categorical_accuracy: 0.5161
Epoch 2/50
7200/7200 [==============================] - 6s 901us/step - loss: 1.3147 - categorical_accuracy: 0.6268 - val_loss: 1.4035 - val_categorical_accuracy: 0.5978
Epoch 3/50
7200/7200 [==============================] - 7s 903us/step - loss: 0.9920 - categorical_accuracy: 0.7210 - val_loss: 1.2969 - val_categorical_accuracy: 0.6150
Epoch 4/50
7200/7200 [==============================] - 7s 909us/step - loss: 0.8085 - categorical_accuracy: 0.7815 - val_loss: 1.2476 - val_categorical_accuracy: 0.6178
Epoch 5/50
7200/7200 [==============================] - 6s 892us/step - loss: 0.6741 - categorical_accuracy: 0.8244 - val_loss: 1.2123 - val_categorical_accuracy: 0.6311
Epoch 6/50
7200/7200 [==============================] - 7s 910us/step - loss: 0.5770 - categorical_acc

7200/7200 [==============================] - 8s 1ms/step - loss: 0.0630 - categorical_accuracy: 0.9876 - val_loss: 1.5070 - val_categorical_accuracy: 0.6306
Epoch 50/50
7200/7200 [==============================] - 7s 1ms/step - loss: 0.0616 - categorical_accuracy: 0.9871 - val_loss: 1.5119 - val_categorical_accuracy: 0.6322


In [74]:
score = model.evaluate(x_test, y_test, batch_size = batch_size, verbose = True)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])


1000/1000 [==============================] - 0s 209us/step

Test categorical_crossentropy: 1.3015312871932982
Categorical accuracy: 0.656
